In [10]:
from qiskit.quantum_info import SparsePauliOp, Statevector

def build_cost_h(n):
    paulis = ['I' * i + 'Z' + 'I' * (n - i - 1) for i in range(n)]
    coeffs = [-1.0] * n
    return SparsePauliOp.from_list(list(zip(paulis, coeffs)))

def build_driver_h(n):
    paulis = ['I' * i + 'X' + 'I' * (n - i - 1) for i in range(n)]
    coeffs = [-1.0] * n
    return SparsePauliOp.from_list(list(zip(paulis, coeffs)))

def build_comm_h(n):
    paulis = ['I' * i + 'Y' + 'I' * (n - i - 1) for i in range(n)]
    coeffs = [2.0] * n
    return SparsePauliOp.from_list(list(zip(paulis, coeffs)))

from qiskit.circuit import QuantumCircuit
from numpy import pi
n_qubits = 4
cost_h = build_cost_h(n_qubits)
driver_h = build_driver_h(n_qubits)
comm_h = build_comm_h(n_qubits)

print(cost_h)
print(driver_h)
print(comm_h)

# ----------- Pauli Gate 演化 -------------
def apply_pauli_evolution(qc, pauli_str, angle):
    n = qc.num_qubits
    pauli_str = pauli_str[::-1]  # Qiskit 是 little-endian
    indices = [i for i, p in enumerate(pauli_str) if p != 'I']
    if not indices:
        return

    for i in indices:
        if pauli_str[i] == 'X':
            qc.h(i)
        elif pauli_str[i] == 'Y':
            qc.sdg(i)
            qc.h(i)

    if len(indices) > 1:
        for i in range(len(indices) - 1):
            qc.cx(indices[i], indices[i + 1])
        target = indices[-1]
    else:
        target = indices[0]

    qc.rz(2 * angle, target)

    if len(indices) > 1:
        for i in reversed(range(len(indices) - 1)):
            qc.cx(indices[i], indices[i + 1])

    for i in indices:
        if pauli_str[i] == 'X':
            qc.h(i)
        elif pauli_str[i] == 'Y':
            qc.h(i)
            qc.s(i)


# ----------- 单层 FALQON 结构 -------------
def apply_falqon_layer(qc, beta_k, cost_h, driver_h, delta_t):

    for label, coeff in zip(driver_h.paulis.to_labels(), driver_h.coeffs):
        apply_pauli_evolution(qc, label, float(coeff.real * beta_k * delta_t))
    for label, coeff in zip(cost_h.paulis.to_labels(), cost_h.coeffs):
        apply_pauli_evolution(qc, label, float(coeff.real * beta_k * delta_t))

from qiskit import transpile


def falqon_qiskit_fast_final_gate_count(n_qubits, n_layers, beta_1, delta_t, cost_h, driver_h, comm_h, 
                                        target_energy=-1.13728383, energy_tol=0.001):
    beta = [beta_1]
    energies = []

    state = Statevector.from_label("+" * n_qubits)
    final_circuit = QuantumCircuit(n_qubits)
    final_circuit.h(range(n_qubits))  # 初始 H^{⊗n}

    for i in range(n_layers):
        qc_layer = QuantumCircuit(n_qubits)
        apply_falqon_layer(qc_layer, beta[i], cost_h, driver_h, delta_t)

        final_circuit.compose(qc_layer, inplace=True)
        state = state.evolve(qc_layer)

        energy = state.expectation_value(cost_h).real
        next_beta = -delta_t * state.expectation_value(comm_h).real

        energies.append(energy)
        beta.append(next_beta)

        print(f"第{i + 1}步：能量 = {energy:.8f} Ha | β_{i} = {beta[i]:+.6f}")


        # ✅ 判断是否达到误差容限
     

    return beta, energies, final_circuit


from qiskit import transpile

# 基础门集合（例如 IBM 超导平台）

# 执行主算法
basis = ["ecr", "id", "rz", "sx", "x"]
basis2= ['rz', 'rx', 'cx']  # IBM 超导平台的典型基础门集
n_layers = 1000
beta_1 = 0.03
delta_t = 0.03
res_beta, res_energies, final_circuit = falqon_qiskit_fast_final_gate_count(
    n_qubits=4,
    n_layers=n_layers,
    beta_1=beta_1,
    delta_t=delta_t,
    cost_h=cost_h,
    driver_h=driver_h,
    comm_h=comm_h
)
#print(final_circuit.draw())

from qiskit.transpiler import CouplingMap
coupling_map = CouplingMap.from_full(4)
from qiskit.transpiler import CouplingMap

# 定义线性拓扑：0–1–2–3
linear_coupling = CouplingMap(couplinglist=[(0, 1), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2)])
transpiled_circuit= transpile(
    final_circuit,
    basis_gates=basis2,  # 典型 IBM 超导平台的基础门集合
    coupling_map=linear_coupling,  # 线性拓扑
    optimization_level=1
)
for gate, count in transpiled_circuit.count_ops().items():
    print(f"{gate}: {count}")

# ✅ 最后统一 transpile 并统计基础门数量
#transpiled_circuit = transpile(final_circuit, basis_gates=basis, optimization_level=1)
final_gate_counts = transpiled_circuit.count_ops()

# ✅ 打印统计
print("\n📊 最终电路在指定基础门集下的门数量统计：")
for gate, count in final_gate_counts.items():
    print(f"{gate}: {count}")




SparsePauliOp(['ZIII', 'IZII', 'IIZI', 'IIIZ'],
              coeffs=[-1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j])
SparsePauliOp(['XIII', 'IXII', 'IIXI', 'IIIX'],
              coeffs=[-1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j])
SparsePauliOp(['YIII', 'IYII', 'IIYI', 'IIIY'],
              coeffs=[2.+0.j, 2.+0.j, 2.+0.j, 2.+0.j])
第1步：能量 = 0.00000000 Ha | β_0 = +0.030000
第2步：能量 = -0.00000019 Ha | β_1 = +0.000432
第3步：能量 = -0.00000038 Ha | β_2 = +0.000438
第4步：能量 = -0.00000058 Ha | β_3 = +0.000445
第5步：能量 = -0.00000078 Ha | β_4 = +0.000451
第6步：能量 = -0.00000099 Ha | β_5 = +0.000457
第7步：能量 = -0.00000120 Ha | β_6 = +0.000464
第8步：能量 = -0.00000143 Ha | β_7 = +0.000471
第9步：能量 = -0.00000165 Ha | β_8 = +0.000477
第10步：能量 = -0.00000189 Ha | β_9 = +0.000484
第11步：能量 = -0.00000213 Ha | β_10 = +0.000491
第12步：能量 = -0.00000238 Ha | β_11 = +0.000498
第13步：能量 = -0.00000263 Ha | β_12 = +0.000506
第14步：能量 = -0.00000290 Ha | β_13 = +0.000513
第15步：能量 = -0.00000317 Ha | β_14 = +0.000520
第16步：能量 = -0.00000345 Ha | β_15 = +0.000528